## Visualización y descarga de un mosaico de coleccion Sentinel-2

## 1. Libraries

In [ ]:
# run if geemap is not installed
pip install geemap

In [ ]:
# Instalar geemap package
import ee
ee.Initialize()
import geemap as emap
Map = emap.Map()
Map.add_basemap('SATELLITE')

## 2. Codes

In [47]:
# study area
#region = ee.FeatureCollection('projects/map-cover-huanuco/assets/landscape_quishqui')
#region = ee.FeatureCollection('projects/map-cover-huanuco/assets/landscape_amarilis')
region = ee.FeatureCollection('projects/map-cover-huanuco/assets/landscape_molinos_p2')

# Define collection
colS2 = 'COPERNICUS/S2_SR_HARMONIZED'

# masking clouds
def maskSQA(image):
    qa = image.select('QA60')
    opaque_cloud = (1 << 10)
    cirrus_cloud = (1 << 11)
    mask = qa.bitwiseAnd(opaque_cloud).eq(0).And(qa.bitwiseAnd(cirrus_cloud).eq(0))
    return image.updateMask(mask)

# bands names
bandNames = ['B2_median','B3_median','B4_median','B8_median','B8A_median','B11_median','B12_median'] 
#blue, red, green, red, red-edge, nir y swir1

img = ee.ImageCollection(colS2)\
                .filterDate('2022-08-01','2022-10-20')\
                .filterBounds(region)\
                .filter(ee.Filter.lt('CLOUDY_PIXEL_PERCENTAGE',10))\
                .map(maskSQA)\
                .reduce(ee.Reducer.median()).select(bandNames)\
                .clip(region)

# Definimos los parametros de visualizacion
vizParams = {
  'bands': ['B11_median', 'B8_median', 'B4_median'],
  'min': 0,
  'max': 3500,
  'gamma': [0.95, 1.1, 1]
}

Map.centerObject(region, 10)
Map.addLayer(img, vizParams, 'S2')
Map.addLayer(region, {}, 'default display', True, opacity = 0.2)
Map

Map(bottom=1108084.0, center=[-10.095389044502927, -75.96588340234555], controls=(WidgetControl(options=['posi…

## 3. Download images

In [ ]:
# Descargar directamente a Google Drive
task = ee.batch.Export.image(img, 
                              'S2_2022_09_02_Molinos_P2', 
                              {'region': region.geometry(),
                               'scale': 10,
                               'maxPixels': 1e13,
                               'folder': 'CLASES_UNI',
                               'fileFormat': 'GeoTIFF',
                               'crs': 'EPSG:4326'})
task.start()